![figure](lab4/politecnico_h-01.png)

# **Programmable Electronics**
#### Bachelor's Degree in Electrical and Computer Engineering

## Laboratory Work 4 - Finite State Machines

-----------------------


## Introduction ##
In this work we will implement a set of state machines using a behavioral description in VHDL, which allows implementing medium-scale circuits using a higher level of abstraction. This document does not provide details on how to use Vivado. Whenever you have questions at this level, consult the Lab1 statement.


### Objectives ###

In this work we demonstrate the implementation of two state machines and a test circuit. These machines implement essential circuits in any synchronous design, and they also serve as a way for students to observe how they can describe state machines in VHDL:
* Rising edge detector.
* Debouncing circuit.
* Edge test circuit with debouncing.


In the instructions below  **{sources}** refers to `C:\Xilinx\EEC_EP\sources` and **{labs}** refers to `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Step 1 - Rising Edge Detector ##

### Step 1.1 ### 

A rising edge detector is a circuit that generates a pulse of one clock cycle duration (we can call it a tick) whenever it detects a transition from '0' to '1' in a given input signal. This circuit can be designed as either a Moore machine or a Mealy machine. Here, we will analyze both solutions to assess their differences.

The state and flow diagrams of the Moore machine are shown in the following figure. The zero and one states indicate the input logic level, while the edge state indicates that a rising edge has occurred, as shown in the timing diagram.


<div>
<img src="lab4/fig1.png" width="900"/>
</div>

If we implement the circuit as a Mealy machine, there are only two states, state **zero** and state **one** , as shown in the state and flow diagrams in the Figure below. When we are in state **zero** and the input changes to '1', the output **tick** is immediately asserted. As soon as the machine stabilizes in state **one**, the output is set back to '0'. The timing diagram of this implementation is also shown in the Figure.


<div>
<img src="lab4/fig2.png" width="800"/>
</div>


Note that this machine is more compact (fewer states) and responds faster than the Moore machine. The Mealy machine responds as soon as the transition is detected, while the Moore machine only responds at t1. As disadvantages, the Mealy machine is more sensitive to disturbances (glitches) in the input signal and the pulse in the output signal does not have a predefined width. However, if the downstream circuits are synchronous with the rising edge of the same clock signal there is no problem since the pulse in the tick output is stable around this edge. As long as this condition is met, the Mealy machine is generally preferred for this application, due to its greater speed. Also in this figure, we can see the corresponding hardware implementation (a flip-flop, an AND gate and an inverter).


* Start by downloading the provided source files (Github or Moodle) and saving them in `{sources}/lab4`.


* Open Vivado from the icon on your desktop or:  `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Select **Create Project** to start the wizard. In the *Create A New Vivado Project* dialog box , select **Next**.


* Click the **Browse** button in the project location field of the new project form, navigate to 'c:/Xilinx/EEC_EP/labs', and click **Select**. Type **lab4** in the project name field. Make sure the *Create Project Subdirector*y box is checked. Click *Next**.


* Select the **RTL Project** option in the *Project Type* form and click **Next**. Make sure the *Do not specify sources at this time* box is checked.


* In the *Default Part* form, select the Boards option and choose the **Basys3** board. Click **Next** and then **Finish** to create the Vivado project.


* In the **Sources** panel, select *Add Sources* and follow the necessary steps (**Add or create design sources**) to add the **edge_detect.vhd** file, which contains the VHDL description of different architectures of the edge detector described above. The file should be added with the *Copy sources* into project option. 


* Open the file in Vivado, look at the code. Analyze the different architectures and if you have any questions, ask your teacher.



### Step 1.2 ### 

In this section, we will look at how the different architectures are implemented by Vivado

* Open the file **edge_detect.vhd** and check which one is active (the other two are commented out).


* Choose the *Open Elaborated Design* operation and open the schematic for this circuit. Note that the schematic includes two state registers, a set of multiplexers that implement the next-state logic, and a multiplexer that implements the output logic (remember that multiplexers can be used to implement logic circuits).

* Close the *Elaborated Design* view in the blue bar.


* *Run Synthesis* and open the synthesized project. Open the schematic corresponding to this circuit and check the differences between the designed circuit and a synthesized circuit. Note that now there are input buffers (IBUF), a global buffer (BUFG) for the clock signal, the logic is mapped into Look-up-tables (LUTs) and the registers are mapped into D-type flip-flops with clear and enable (FDCE).


* Look at the *Project Summary* window. Check how many resources are used in this implementation. It should have 2 flip-flops, 2 LUTs, 4 I/O buffers and a global buffer.


* Close the *Synthesized Design* view in the blue bar, go to the VHDL file and activate the Mealy Machine architecture (comment out the Moore Machine architecture). Click **Save**.


* Repeat the previous steps to analyze the Mealy Machine architecture. Note that you should now have a single state register and that the number of resources is smaller.


* Close the *Synthesized Design* view in the blue bar and repeat the previous paragraph for the *Gate Level* architecture. Note the differences in the schematic in RTL and post-synthesis. In RTL, exactly the logic circuits described in VHDL appear (and not generic multiplexers), but they are mapped to LUTs and FDCE in the same way. The resources used in this solution are the smallest of all the solutions analyzed in this section.


* Close the *Synthesized Design* view in the blue bar.

* Re-enable the Moore architecture in the **edge_detect.vhd** file and comment out the rest.

______

## Step 2 - Debouncing circuit ##

### Step 2.1 ### 

The switches and pushbuttons on the development board are mechanical devices. As such, they may exhibit vibration (bounce) when pressed. The result of this vibration is the appearance of small unwanted pulses (glitches), typically lasting around 20ms.

The purpose of a debounce circuit is to eliminate these glitches. Depending on its implementation, we can have two types of response, illustrated in the following figure.

<div>
<img src="lab4/fig3.png" width="500"/>
</div>

In this work, we will implement the first scheme using a State Machine and a clock generator with Tclk=10ms. The state machine must ignore glitches and change the output value only when the input has stabilized (between 20ms and 30ms). The state diagram is represented in the figure below. The states **zero** and **one** indicate that the input signal has stabilized with the logical value '0' and '1', respectively. The states **waitx_1**, **waitx_2** and **waitx_3** ensure that the output is only changed when the input is stable for at least 2 counter cycles (if the counter is close to the end of counting when the machine goes to state wait1_1 , the waiting time will be close to 20ms, otherwise it will be close to 30ms).


<div>
<img src="lab4/fig4.png" width="700"/>
</div>


* The VHDL description of this circuit is provided in the file **db_fsm.vhd**. Add it to the Vivado project (*Add or create design sources*), open the file, and look at the code.

* Analyze the code and verify that it is in accordance with the state diagram in the figure above. Note that the value of the generic N is equal to twenty (N=20). Try to understand why.

* To simulate, synthesize or implement this module, you must first right-click on the module (in the *Sources* window), and then choose the **Set as Top** option that appears in the dialog window. This option allows you to define the module as the circuit to be considered in the following actions.

* Add the stimulus file **db_fsm_tb.vhd** and simulate its operation. Try to understand why the value of the generic N was changed when the module was instantiated in the testbench. Also try to understand the waiting times between changes in the input stimulus values (**sw**).

* Add the internal signals **state_reg** and **state_next** to the simulator's waveform window. To do this, select the **UUT** module in the *Instances and Process Name* window and drag the signals from the Simulation Objects for UUT window to the waveform window.

* Observe the simulation results and verify that it works as expected.

* Close the Simulation view in the blue strip.


______________

## Step 3 - Rising edge test circuit, with debouncing ##

### Step 3.1 ### 

To test the effectiveness of the circuits implemented in sections 1 and 2, we will use a bounce counting circuit. The block diagram is shown in the figure below. Note that the clock line (net clk) is not fully represented so as not to complicate the diagram. By definition, the clock inputs of the modules of a synchronous circuit are connected to the clock signal, which is unique. Therefore, it is not necessary to draw them (the connection is assumed to be obvious). This way the schematic is simpler, which increases its clarity.



<div>
<img src="lab4/fig5.png" width="900"/>
</div>


In the lower branch, the signal from the pushbutton passes through the debouncing circuit and the edge detector before attacking an 8-bit counter. Thus, each time the button is pressed, only one pulse should be generated and the counter should increment only one unit. In the upper branch, the lack of a debouncing circuit should cause the counter to also count glitches caused by mechanical vibration.


* Add the **db_test.vhd** file to your project, as well as the **disp_hex_mux.vhd** module, already used in previous work. Note that the **db_test** file instantiates the debounce module and implements (at Gate Level) two edge detector circuits. Analyze the VHDL code.

* Select the **db_test.vhd** file and ensure that it is the top file in the Sources window .

* Follow the usual procedures to synthesize and implement the circuit (Synthesize and Implement Design processes). Remember that it is necessary to include the Basys3 pin file, duly modified, since we are going to implement this module on the development board.

* Check the functionality of the debouncing circuit in Basys3.



_________

## Challenge ##


Using the Lab3 digital stopwatch and the circuits analyzed here, implement a digital clock (seconds and minutes) that allows you to set the initial counting value of each of the digits, using a push button to adjust each one of them. Each time one of the buttons is pressed, the corresponding digit should increase by one unit. This adjustment should be made when the clock is in stop mode (**Go='0'**).

You must first draw the block diagram of the system and only then move on to its implementation in VHDL.
